In [64]:
import collections
import inspect
OD = collections.OrderedDict

def _nargs(f):
    return len(inspect.signature(f).parameters)

def _derr(*args):
    if args[0] is None:
        print("Got NONE argument")
        raise
    else:
        print("ERROR: argument %s is incorrect in some way!" % args[0])
        import pdb; pdb.set_trace()
    # dummy lines to allower user to examine the arg stack without
    # leaving by mistake
    args
    args
    
class NotFound(KeyError):
    """ Signal that variable was not found in the list.
    To be raised by user code as a signal to `compare_lists`. 
    """
    pass

NotFoundErrors = (NotFound, KeyError, IndexError)

def _equal(a,b):
    return a == b

def _skipvar(v):
    return False

def _getvar(l,v):
    return l[v]

def _special(v):
    return None

def _notfound(v):
    return "", "", "", "Not Found"

def _str(v):
    """ Wrapper to allow default argument to be inspectable for debug purposes ... """
    return str(v)

def compare_lists(cf, of, nf, getvar=_getvar, var2str=_str, skipvar=_skipvar, 
                  vequal=_equal, special=_special, notfound=_notfound, out=print, setvar=lambda a,b:None):
    
    """ Fill in callables!!!
    Copy the templates below....
    
    def getvar(l, v):
        return l[v]
    def var2str(vval):
        return str(vval)
    def skipvar(vname):
        return False
    def vequal(vval, vval2):
        return vval == vval2
    def special(vname):
        return False
    def notfound(vname):
        return "", "", "", "Not Found"
    def out(s):
        print(s)
    """
    
    # sanity check the arguments, since this function is somewhat abstract
    assert isinstance(cf, dict), _derr(cf)
    assert isinstance(of, dict), _derr(of)
    assert isinstance(nf, dict), _derr(nf)
    assert callable(getvar) and _nargs(getvar) == 2, _derr(getvar)
    assert callable(var2str) and _nargs(var2str) == 1, _derr(var2str)
    assert callable(skipvar) and _nargs(skipvar) == 1, _derr(skipvar)  
    assert callable(vequal) and _nargs(vequal) == 2, _derr(vequal)
    assert callable(special) and _nargs(special) == 1, _derr(special)
    assert callable(notfound) and _nargs(notfound) == 1, _derr(notfound)
    assert callable(setvar) and _nargs(setvar) == 2, _derr(setvar)

    olines = []
    wvar = len("Variable")  # min width is width of the header
    for name in nf:
        if skipvar(name):
            continue
        cs = os = ns = m = ""
        s = special(name)
        if s is not None and s is not False:
            cs, os, ns, m = s
        else:
            try:
                cv = getvar(cf, name)
                ov = getvar(of, name)
                nv = getvar(nf, name)
            except NotFoundErrors:
                cs, os, ns, m = notfound(name)
            else:
                if vequal(nv, ov) and vequal(ov, cv):
                    cs = os = ns = ""
                    m = "No Change"
                elif not vequal(nv, ov) and vequal(ov, cv):
                    cs = ""
                    os = var2str(ov)
                    ns = var2str(nv)
                    m  = "N    N"
                elif vequal(nv, ov) and not vequal(ov, cv):
                    cs = var2str(cv)
                    os = ""
                    ns = var2str(nv)
                    m  = "N    Y"
                elif vequal(nv, cv) and not vequal(ov, cv):
                    cs = var2str(cv)
                    os = ""
                    ns = var2str(nv)
                    m  = "Y    N"
                elif not vequal(nv, cv) and not vequal(cv, ov):
                    cs = var2str(cv)
                    os = var2str(ov)
                    ns = var2str(nv)
                    m  = "N    N"
                else:
                    # should be unreachable
                    cs = var2str(cv)
                    os = var2str(ov)
                    ns = var2str(nv)
                    m  = "???"
        olines.append((name, cs, os, ns, m))
        wvar = max(wvar, len(name))
            
            # todo - variable column width 
#             wc1  = max(wc1, len(cs))
#             wc2  = max(wc2, len(os))
#             wc3  = max(wc3, len(ns))
#             wc4  = max(wc4, len(m))
                
    linefmt = "%%-%ds: %%-10s| %%-10s| %%-10s| %%-10s| " % (wvar + 1)
    w1 = 10
    w2 = 10
    w3 = 10
    w4 = 10
    linefmt = "%%-%ds: %%-%ds| %%-%ds| %%-%ds| %%-%ds|  " % (wvar + 1, w1, w2, w3, w4)
    header1 = "%-*s  %-*s  %-*s  %-*s  %-*s  " % (wvar + 1, "", w1, "", w2, "Contents", w3, "", w4, "Matches") + "Restore"
    header2 = linefmt%("Variable", "Cust File", "Old File", "New File", "USR  PBS") + "Action"
    header3 = len(header2) * "-"

    out(header1)
    out(header2)
    out(header3)
    for l in olines:
        out(linefmt%l)
            

In [65]:
from jpnotebooks.Sessions.cfg_merge.sys_vars import _parse_sysvars
from jpnotebooks.Sessions.cfg_merge.lv_parse import flatten as lv_flatten

def compare_sysvars(cff, off, nff):
    """
    Since the naming is confusing in this function
    and compare_sysvars2, let me explain:
    
    There are 4 levels of xxxf variables:
        xxxf (parameter) -> dict of all files in folder
         xff             -> filename of sys vars file
         xLV             -> parsed contents of file (LVType)
          xf             -> flattened, parsed contents (OrderdDict)
          
    All the comparison functions can follow the same basic template:
     1) callback functions for compare_lists (verify var2str and vequal, they're important!)
     2) take the filename for each file and parse it into a dict of dotted names -> values,
        e.g. Agitation.Minimum(RPM) -> 3. No nesting!
     3) Call compare_lists
     
    Sys vars is a bit special, since the output has to be "XML" and 
    Type information would be lost if we just flattened straight to python values 
    (e.g. I32 vs I16, etc). So, the value passed in the list is the LVType, and the callbacks
    extract the value where appropriate. 
    """
    # callbacks for compare_lists
    def getvar(l, v): return l[v]
    def var2str(vval): return vval.tostr()                         # vvals are LVType, not str!
    def skipvar(vname): return False
    def vequal(vval, vval2): return vval.tostr() == vval2.tostr()  # vvals are LVType, not str!
    def special(vname): return False
    def notfound(vname): return "", "", "", "Not Found"
    def out(s): print(s)
        
    # parse scheme for sys vars file
    def parse_sysvars(ff): return _parse_sysvars(ff)  # was already programmed :)
    def _flatten(LV):
        d = OD()
        for g in LV.val:
            for v in g.val:
                d[g.name+"."+v.name] = v
        return d
            
    # prepare the list (dict) for compare_lists...
    cLV = _parse_sysvars(cff)
    oLV = _parse_sysvars(off)
    nLV = _parse_sysvars(nff)
    
    cf  = _flatten(cLV)
    of  = _flatten(oLV)
    nf  = _flatten(nLV)
    
    compare_lists(cf, of, nf, getvar, var2str, skipvar, vequal, special, notfound, out)
    
def compare_sysvars2(custf, oldf, newf):
    cff = custf['system variables']
    off =  oldf['system variables']
    nff =  newf['system variables']
    return compare_sysvars(cff, off, nff)

In [67]:
# PBS 3 CDI 3
# Example using PBS 3 CDI 3
cff = 'D:\\Customer_Backups\\000311T2902 180102\\Rio_Config\\System Variables.cfg'
off = 'D:\\auto_hd_install\\default configs\\IC3405 Rev C\\Mag 3 150727\\System Variables.cfg'
nff = 'd:\\auto_hd_install\\default configs\\IM00226 Rev B\\Mag 3\\System Variables.sys'
compare_sysvars(cff, off, nff)

                                                       Contents                Matches     Restore
Variable                                 : Cust File | Old File  | New File  | USR  PBS  |  Action
--------------------------------------------------------------------------------------------------
Temperature.P Gain (%/C)                 :           | 40.00000  | 50.00000  | N    N    |  
Temperature.I Time (min)                 :           | 6.00000   | 26.00000  | N    N    |  
Temperature.D Time (min)                 :           |           |           | No Change |  
Temperature.Alpha                        :           |           |           | Not Found |  
Temperature.Beta                         :           |           |           | Not Found |  
Temperature.Gamma                        :           |           |           | Not Found |  
Temperature.Linearity                    :           |           |           | Not Found |  
Temperature.Heat Manual Max (%)          :          

In [72]:
import collections
import inspect
OD = collections.OrderedDict
from jpnotebooks.Sessions.cfg_merge.sys_vars import _parse_sysvars

class NotFound(KeyError):
    """ Signal that variable was not found in the list.
    To be raised by user code as a signal to `compare_lists`. 
    """
    pass

NotFoundErrors = (NotFound, KeyError, IndexError)

def _update(var, val):
    pass

def merge_lists(cf, of, nf, getvar, skipvar, vequal, notfound, setvar):
    for name in nf:
        if skipvar(name):
            continue
        nv = getvar(nf, name)
        try:
            cv = getvar(cf, name)
            ov = getvar(of, name)
        except NotFoundErrors:
            v = notfound(name)
            if v is None:
                v = nv
        else:
            if vequal(nv, ov) and vequal(ov, cv):
                v = nv
            elif not vequal(nv, ov) and vequal(ov, cv):
                v = nv                
            elif vequal(nv, ov) and not vequal(ov, cv):
                # XXX Conflict
                v = cv
            elif vequal(nv, cv) and not vequal(ov, cv):
                v = nv
            elif not vequal(nv, cv) and not vequal(cv, ov):
                # XXX Conflict
                v = cv
            else:
                # should be unreachable
                raise NameError("Unreachable code for %r"%name)
        setvar(name, v)
    for name in cf:
        if name not in nf:
            notfound(name)
        
        
def merge_sysvars_inner(cf, of, nf, ignore, force):
    ign = frozenset(ignore)
    # Callbacks
    def getvar(f,name):
        return f[name]
    def skipvar(v):
        return v in ign
    def vequal(a,b):
        return a.val == b.val
    def notfound(n):
        if n not in nf:
            print("Old Name: %s"%n)
        return None
    
    def setvar(name, v):
        # only message if value is updated
        if name in force:
            print("Forcing %-45s: %.5f -> %.5f"%(name, nf[name].val, force[name]))
            nf[name].val = force[name]
        elif nf[name].val != v.val:
            print("Updating %-45s: %.5f -> %.5f"%(name, nf[name].val, v.val))
            nf[name] = v
    
    return merge_lists(cf, of, nf, getvar, skipvar, vequal, notfound, setvar)
    



In [73]:
def merge_sysvars(cff, off, nff, outf=None, ignore=(), force=None):
    """
    1. Parse
    2. Flatten ("Group.Varname" = "Value")
    3. Merge
    4. Output
    """
    def parse_sysvars(ff): 
        return _parse_sysvars(ff)
    
    def _flatten(LV):
        d = OD()
        for g in LV.val:
            for v in g.val:
                d[g.name+"."+v.name] = v
        return d
            
    cLV = _parse_sysvars(cff)
    oLV = _parse_sysvars(off)
    nLV = _parse_sysvars(nff)
    
    cf  = _flatten(cLV)
    of  = _flatten(oLV)
    nf  = _flatten(nLV)
    merge_sysvars_inner(cf, of, nf, ignore, force or {})

    if outf:
        with open(outf, 'w') as f:
            f.write(nLV.toxml())

In [74]:
# Example using PBS 3 CDI 3
cff = 'D:\\Customer_Backups\\000311T2902 180102\\Rio_Config\\System Variables.cfg'
off = 'D:\\auto_hd_install\\default configs\\IC3405 Rev C\\Mag 3 150727\\System Variables.cfg'
nff = 'D:\\auto_hd_install\\default configs\\IM00226 Rev B\\Mag 3\\System Variables.sys'
merge_sysvars(cff, off, nff, "test.xml") #, force={"Agitation.Minimum (RPM)": 5}) #, ["Agitation.Minimum (RPM)"])

Updating Agitation.Minimum (RPM)                      : 3.00000 -> 2.00000
Updating pH.CO2 Auto Max (%)                          : 100.00000 -> 7.50000
Updating pH.Base Manual Max (%)                       : 50.00000 -> 0.00000
Updating pH.Base Auto Max (%)                         : 50.00000 -> 0.00000
Updating pH.Deadband                                  : 0.02000 -> 0.01000
Updating DO.N2 I Time (min)                           : 50.00000 -> 1.00000
Updating Gas Data.Mismatch Thresh (V)                 : 0.10000 -> 0.20000
Updating Gas Data.Manual Max (LPM)                    : 0.50000 -> 1.00000
Updating Process Alarms.Agitation Low Low (RPM)       : 10.00000 -> 30.00000
Updating Process Alarms.Agitation Low (RPM)           : 15.00000 -> 32.00000
Updating Process Alarms.Agitation High (RPM)          : 35.00000 -> 38.00000
Updating Process Alarms.Agitation High High (RPM)     : 38.00000 -> 40.00000
Updating Process Alarms.Temp High (C)                 : 38.00000 -> 37.10000
Updating P

In [69]:
import configparser

c = configparser.ConfigParser()

c.read_file("test.sysvars.patch")